In [1]:
cd ..

/Users/amiyaguchi/wikipedia-retention


In [2]:
! ls data/processed/

2007-1-enwiki-projection-user-dev.csv
2007-1-enwiki-projection-user-roles.csv
2007-1-enwiki-projection-user.csv
2007Q1-user-network-v1.csv
2007Q1-user-network-v2.csv
2007Q1-user-roles-v2.txt
admin_mapping.csv
admins.csv
all_article_features.csv
all_user_features.csv
article-network-intermediate-block-articleset
avg_role_features
base_features_reg.csv
community_norm_features.csv
enwiki-meta-compact
enwiki-projection-user-dev.csv
kcore-2007-1.csv
nodesense-v1.csv
rolx-nmf-G.csv
rolx-roles
rolx-v
user-network-v3-mappings
user-network-v3-v
user-network-v3.csv


In [7]:
# Model for neural net regression on all 
import numpy as np
import pandas as pd
import sklearn

# load features
base = "data/processed/"

user_df = pd.read_csv(base + 'base_features_reg.csv', header=None)
role_df = pd.read_csv(base + 'rolx-nmf-G.csv', header=None, skiprows=1)
article_df = pd.read_csv(base + 'all_article_features.csv', header=None)

y = user_df.iloc[:, -1].values

user_df = user_df.drop([1,user_df.columns[-1]], axis=1)

In [26]:
# process joined data
X = (
    user_df
    .merge(article_df, how='left', on=0)
    #.merge(role_df, how='left', on=0)
    .fillna(0.0)
).values

In [27]:
from sklearn.preprocessing import MinMaxScaler

X = np.ndarray.astype(X[:,1:],float) # remove user_id
X[np.isnan(X)] = 0 # clear NaNs
# add new column for log(sum(log(textdata)))
lslt = np.array([np.log(X[:,5])+1]).T
X = np.append(X, lslt, 1)

# min-max scaling

scalar = MinMaxScaler(feature_range=(0,1))
scalar.fit(X)
dmin = scalar.data_min_
dmax = scalar.data_max_
Xnorm = scalar.transform(X)
Xnorm = Xnorm - Xnorm.mean(axis=0)

yl = np.log(y+1) # run on log y for smoother fit

In [28]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.metrics import log_loss, r2_score
import matplotlib.pyplot as plt


# train model
def fit_model(model,X,y):
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=115)
    model.fit(X_train, y_train)
    score = model.score(X_test,y_test) # can have sample weight here
    return model, score

def plot_preds(preds, y, xlab='prediction', ylab='actual contribution'):
    plt.plot(np.exp(preds-1),np.exp(y-1),'.')
    mx = np.exp(min(np.max(preds),np.max(y)))
    plt.plot([1,mx],[1,mx],color='red')
    plt.xscale('log')
    plt.yscale('log')
    plt.xlabel(xlab)
    plt.ylabel(ylab)
    plt.show()

def plot_yl(yl):
    plt.hist(yl,log=True)
    plt.xlabel('log contribution')
    plt.ylabel('bin count')
    plt.show()

# NOTE: Below is the full 2-model (class * reg)
# alphas = [0.1, 1, 10, 100]
nodes = [(3), (5), (7), (9), (8, 3), (10, 3), (12, 4), (14, 5)]
alphas = [0.1]
# nodes = [(8,3)]
params = {"alpha" : alphas, "hidden_layer_sizes" : nodes}
theta_idx = yl > 0
yl_theta = yl[theta_idx]
X_theta = Xnorm[theta_idx] # note: don't rescale since we need to combine models

# run regression model
MLPR = MLPRegressor(activation = 'relu', solver = 'adam', random_state = 112358)
GSR = GridSearchCV(MLPR, params, return_train_score = True, n_jobs=4)
reg_model, reg_score = fit_model(GSR, X_theta, yl_theta)
rm = reg_model.best_estimator_
reg_preds = rm.predict(Xnorm) # predict on all

# run classification model
MLPC = MLPClassifier(activation='relu', solver='adam', random_state = 112358)
GSC = GridSearchCV(MLPC, params, return_train_score = True, n_jobs=4)
theta = np.ndarray.astype(theta_idx,int)
class_model, class_score = fit_model(GSC, Xnorm, theta)
cm = class_model.best_estimator_
class_preds = cm.predict_proba(Xnorm)[:,1]

# now combine
combined_preds = class_preds * reg_preds
combined_score = sklearn.metrics.r2_score(yl,combined_preds)

# plot class hist
def plot_class_hist(theta_idx, class_preds):
    plt.hist([1-class_preds[theta_idx],class_preds[~theta_idx]],label=['theta=1','theta=0'],log=True,bins=20)
    plt.xlabel("Classification error")
    plt.ylabel("Bin Count")
    plt.legend()
    plt.show()


/Users/amiyaguchi/wikipedia-retention/venv3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/amiyaguchi/wikipedia-retention/venv3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


In [29]:
# without roles
print(reg_score, class_score, combined_score)
# 0.3644307322834125 0.8623086114977545 0.37620858386813083

0.36574926519220075 0.8624038133256297 0.37943924838699383


In [25]:
# with roles
print(reg_score, class_score, combined_score)
# 0.589164057706867 0.8864176537017097 0.5713924962545209

0.589164057706867 0.8864176537017097 0.5713924962545209
